In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'pandas'

In [6]:
df = pd.read_csv("/Users/Mostafa/Desktop/first_year_Pisa/ML/project_implementation/data/monk2/monks-2.train", names=[0,1,2,3,4,5,6,"index"], delimiter= " ")
df.set_index("index", inplace=True)
df

NameError: name 'pd' is not defined

In [ ]:
y = df.iloc[:, 0]  # First column as target
X = df.iloc[:, 1:]  # All other columns as features

# Print the results to verify
print("Target (y):")
print(y.head())
print(len(y))
print("\nFeatures (X):")
print(X.head())

NameError: name 'df' is not defined

In [ ]:
test_data = pd.read_csv("/Users/Mostafa/Desktop/first_year_Pisa/ML/project_implementation/data/monk2/monks-2.test", names=[0,1,2,3,4,5,6,"index"], delimiter= " ")
test_data.set_index("index", inplace=True)
# test_data.head()

In [ ]:
y_test = test_data.iloc[:, 0]
X_test = test_data.iloc[:, 1:]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shape of the resulting datasets
print("Training Features Shape:", X_train.shape)
print("Validation Features Shape:", X_val.shape)
print("Training Target Shape:", y_train.shape)
print("Validation Target Shape:", y_val.shape)

Training Features Shape: (135, 6)
Validation Features Shape: (34, 6)
Training Target Shape: (135,)
Validation Target Shape: (34,)


In [8]:
import numpy as np
# from cd import *
np.random.seed(0)

    
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
   
    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues) 
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values 
        self.dinputs = np.dot(dvalues, self.weights.T)         

class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Since we need to modify the orginial variable,
        # let's make a copy of the values first
        self.dinputs = dvalues.copy()
        
        # zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

        
class Activation_Softmax:
    def forward(self, inputs):
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        
        for index, (single_output, single_dvalues) in \
                enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            
            jacobian_matrix = np.diagflat(single_output) - \
                                np.dot(single_output, single_output.T)

            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)

        if len(y_true.shape) == 1:
            correct_confidence = y_pred_clipped[range(samples), y_true]

        elif len(y_true.shape) == 2:
            correct_confidence = np.sum(y_pred_clipped * y_true, axis=1) 

        negative_log_likelihoods = np.log(correct_confidence)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):

        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples


class Activation_Softmax_Loss_CategoricalCrossentropy():

    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()


    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)

        # Set the output
        self.output = self.activation.output

        return self.loss.calculate(self.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)

        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)

        self.dinputs = dvalues.copy()

        self.dinputs[range(samples), y_true] -= 1

        self.dinputs = self.dinputs / samples



class Optimizer_SGD:
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate 
        self.current_learning_rate = learning_rate 
        self.decay = decay 
        self.iterations = 0 
        self.momentum = momentum 
        
    def pre_update_params(self):
        if self.decay: 
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))
                
                                
    def update_params(self, layer):
        
         # If we use momentum 
        if self.momentum: 
 
            # If layer does not contain momentum arrays, create them 
            # filled with zeros 
            if not hasattr(layer, 'weight_momentums'): 
                layer.weight_momentums = np.zeros_like(layer.weights) 
                # If there is no momentum array for weights 
                # The array doesn't exist for biases yet either. 
                layer.bias_momentums = np.zeros_like(layer.biases) 
 
            # Build weight updates with momentum - take previous 
            # updates multiplied by retain factor and update with 
            # current gradients 
            weight_updates = \
                self.momentum * layer.weight_momentums - \
                self.current_learning_rate * layer.dweights 
            layer.weight_momentums = weight_updates 
 
            # Build bias updates 
            bias_updates = \
                self.momentum * layer.bias_momentums - \
                self.current_learning_rate * layer.dbiases 
            layer.bias_momentums = bias_updates 
 
        # Vanilla SGD updates (as before momentum update) 
        else:
            weight_updates = -self.current_learning_rate * \
                layer.dweights 
            bias_updates = -self.current_learning_rate * \
                layer.dbiases 
 
        # Update weights and biases using either 
        # vanilla or momentum updates 
        layer.weights += weight_updates 
        layer.biases += bias_updates 
            
            
    def post_update_params(self):
        self.iterations += 1
        
# X, y = create_data(100, 3)
 
dense1 = Layer_Dense(6, 64)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(64, 2)

loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

optimizer = Optimizer_SGD(learning_rate=1, decay=1e-3, momentum=0.99)


for epoch in range(10001):
        
    dense1.forward(X_train)

    activation1.forward(dense1.output)

    dense2.forward(activation1.output)

    # print(activation2.output)
    loss = loss_activation.forward(dense2.output, y_train)

    # print(loss_activation.output[:5])

    # print("Loss: ", loss)

    predictions = np.argmax(loss_activation.output, axis=1)

    if len(y.shape) == 2:
        y = np.argmax(y_train, axis=1)
    accuracy = np.mean(predictions==y_train)


    if not epoch % 100:
        print(f"epoch:  {epoch}, " + 
              f"acc: {accuracy:.3f}, " +
              f"loss: {loss:.3f}, " 
              f"learning_rate: {optimizer.current_learning_rate} ")
        
    # Backward pass
    loss_activation.backward(loss_activation.output, y_train)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases 
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()
    

    
        

NameError: name 'X_train' is not defined